In [1]:
# boas práticas: criar uma célula com todas as bibliotecas necessárias
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Turmas 2017.2
# http://dados.ufrn.br/dataset/turmas

# turmas
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'

# matrículas em componentes em 2017.2
notas_url = 'http://bit.do/notas_ufrn_2017_2'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# docentes da ufrn atualizado em 2018
docentes_url = 'http://bit.do/docentes_ufrn_2018'

# turmas 2017.2
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.2
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

docentes = pd.read_csv(docentes_url, sep=';')

In [3]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [4]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57602957,5753061.0,ART2077,LATO SENSU,PRODUÇÃO COREOGRÁFICA I,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T23456
2,57602962,5752714.0,ART2083,LATO SENSU,VIESES ARTÍSTICOS CONTEMPORÂNEOS EM DANÇA ...,DEPARTAMENTO DE ARTES,ABERTA,40.0,1M345 7M2345 17T23456
3,57602963,3031948.0,ART2086,LATO SENSU,PRÁTICAS DE IMPROVISAÇÃO PARA A CENA DAS DANÇA...,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T3456
4,57603192,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
5,57603193,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T456


In [5]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)

In [6]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
turmas_imd_nomesprof.head()

,id_turma,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,57608376,LUCELIO DANTAS DE AQUINO,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,60,5N34 (24/07/2017 - 16/12/2017)
1,57608398,LUCELIO DANTAS DE AQUINO,IMD0027,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II,60,3N34 (24/07/2017 - 16/12/2017)
2,57608691,LUCELIO DANTAS DE AQUINO,IMD0335,GRADUAÇÃO,TÓPICOS ESPECIAIS I,40,35T56 (24/07/2017 - 16/12/2017)
3,57608377,LOURENA KARIN DE MEDEIROS ROCHA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)
4,57608377,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)


In [7]:
# filtrar apenas as turmas do IMD
notas_imd = notas[notas.id_turma.isin(turmas_imd_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_merge = pd.merge(notas_imd, turmas_imd_nomesprof, on='id_turma')

In [9]:
def aprovado(column):
    if column["descricao"]==("APROVADO" or "APROVADO POR NOTA"):
        return 1
    else:
        return 0

In [10]:
# Colocando uma flag nos alunos aprovados em 2017.2
rotulos = notas_imd_merge.apply(aprovado, axis = 1)
notas_imd_merge["aprovado"] = rotulos


In [29]:
#Separando e tratando os alunos de graduação em 2017.2
graduacao_imd = notas_imd_merge[notas_imd_merge["nivel_ensino"] != "LATO SENSU"]
graduacao_imd = graduacao_imd[graduacao_imd["descricao"] != "INDEFERIDO"]
graduacao_imd = graduacao_imd[graduacao_imd["descricao"] != "CANCELADO"]
graduacao_imd = graduacao_imd[graduacao_imd["descricao"] != "EXCLUIDA"]
graduacao_imd.nome_x.unique()

array(['INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO',
       'PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO', 'DESIGN DE SOFTWARE II',
       'INTRODUÇÃO À GOVERNANÇA E GESTÃO DE TI', 'ACESSIBILIDADE DIGITAL',
       'EMPREENDEDORISMO',
       'PROCESSO DE DESENVOLVIMENTO DE SOFTWARE EDUCACIONAL',
       'TÓPICOS ESPECIAIS I', 'INTRODUÇÃO AO TESTE DE SOFTWARE',
       'PROGRAMAÇÃO CONCORRENTE E DISTRIBUÍDA',
       'RELAÇÕES ENTRE COMUNICAÇÃO, MÍDIA E EDUCAÇÃO',
       'CÁLCULO DIFERENCIAL E INTEGRAL I', 'SISTEMAS OPERACIONAIS',
       'REDES DE COMPUTADORES', 'VETORES E GEOMETRIA ANALÍTICA',
       'JOGOS DIGITAIS COMO FERRAMENTA PEDAGÓGICA',
       'DESENVOLVIMENTO DE SISTEMAS WEB I',
       'PROTOCOLO E SERVIÇOS DE INTERNET',
       'PROJETOS BASEADOS EM MICROCONTROLADORES',
       'PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL B"',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL C"',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL D"

In [50]:
#Separando as turmas de ITP e os aprovados.
itp =graduacao_imd[graduacao_imd["nome_x"] == "INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO"]
itp = itp[itp.duplicated(subset = "discente", keep = 'first') != True]
aprovados_itp = itp.pivot_table(index = "nome_x", values = "aprovado", aggfunc = np.sum)
quantidade_alunos = itp.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [32]:
#Separando as turmas de PTP e os aprovados.
ptp =graduacao_imd[graduacao_imd["nome_x"] == "PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO"]
ptp = ptp[ptp.duplicated(subset = "discente", keep = 'first') != True]
aprovados_ptp = ptp.pivot_table(index = "nome_x", values = "aprovado", aggfunc = np.sum)
quantidade_alunos = ptp.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [33]:
#Separando as turmas de LIP 1 e os aprovados.
lip1 =graduacao_imd[graduacao_imd["nome_x"] == "LINGUAGEM DE PROGRAMAÇÃO I"]
lip1 = lip1[lip1.duplicated(subset = "discente", keep = 'first') != True]
aprovados_lip1 = lip1.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = lip1.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [34]:
#Separando as turmas de DESIGN DE SOFTWARE II e os aprovados
ds2 =graduacao_imd[graduacao_imd["nome_x"] == "DESIGN DE SOFTWARE II"]
ds2 = ds2[ds2.duplicated(subset = "discente", keep = 'first') != True]
aprovados_ds2 = ds2.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = ds2.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [35]:
#Separando as turmas de INTRODUÇÃO À GOVERNANÇA E GESTÃO DE TI e os aprovados
iggti =graduacao_imd[graduacao_imd["nome_x"] == "INTRODUÇÃO À GOVERNANÇA E GESTÃO DE TI"]
iggti = iggti[iggti.duplicated(subset = "discente", keep = 'first') != True]
aprovados_iggti = iggti.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = iggti.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [36]:
#Separando as turmas de ACESSIBILIDADE DIGITAL e os aprovados
ad =graduacao_imd[graduacao_imd["nome_x"] == "ACESSIBILIDADE DIGITAL"]
ad = ad[ad.duplicated(subset = "discente", keep = 'first') != True]
aprovados_ad = ad.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = ad.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [37]:
#Separando as turmas de EMPREENDEDORISMO e os aprovados
emp =graduacao_imd[graduacao_imd["nome_x"] == "EMPREENDEDORISMO"]
emp = emp[emp.duplicated(subset = "discente", keep = 'first') != True]
aprovados_emp = emp.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = emp.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [38]:
#Separando as turmas de PROCESSO DE DESENVOLVIMENTO DE SOFTWARE EDUCACIONAL e os aprovados
pdse =graduacao_imd[graduacao_imd["nome_x"] == "PROCESSO DE DESENVOLVIMENTO DE SOFTWARE EDUCACIONAL"]
pdse = pdse[pdse.duplicated(subset = "discente", keep = 'first') != True]
aprovados_pdse = pdse.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = pdse.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [39]:
#Separando as turmas de TÓPICOS ESPECIAIS I e os aprovados
te1 =graduacao_imd[graduacao_imd["nome_x"] == "TÓPICOS ESPECIAIS I"]
te1 = te1[te1.duplicated(subset = "discente", keep = 'first') != True]
aprovados_te1 = te1.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = te1.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [ ]:
#Separando as turmas de TÓPICOS ESPECIAIS I e os aprovados
te1 =graduacao_imd[graduacao_imd["nome_x"] == "TÓPICOS ESPECIAIS I"]
te1 = te1[te1.duplicated(subset = "discente", keep = 'first') != True]
aprovados_te1 = te1.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = te1.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [54]:
#Separando as turmas de INTRODUÇÃO AO TESTE DE SOFTWARE e os aprovados
its =graduacao_imd[graduacao_imd["nome_x"] == "INTRODUÇÃO AO TESTE DE SOFTWARE"]
its = its[its.duplicated(subset = "discente", keep = 'first') != True]
aprovados_its = its.pivot_table(index = ["nome_x"], values = "aprovado", aggfunc = np.sum)
quantidade_alunos = its.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_its.aprovado/quantidade_alunos.capacidade_aluno)*100

In [41]:
#Separando as turmas de TÓPICOS ESPECIAIS I e os aprovados
te1 =graduacao_imd[graduacao_imd["nome_x"] == "TÓPICOS ESPECIAIS I"]
te1 = te1[te1.duplicated(subset = "discente", keep = 'first') != True]
aprovados_te1 = te1.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = te1.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [ ]:
#Separando as turmas de TÓPICOS ESPECIAIS I e os aprovados
te1 =graduacao_imd[graduacao_imd["nome_x"] == "TÓPICOS ESPECIAIS I"]
te1 = te1[te1.duplicated(subset = "discente", keep = 'first') != True]
aprovados_te1 = te1.pivot_table(index = ["nome_x"], values = ["aprovado","capacidade_aluno"], aggfunc = np.sum)
quantidade_alunos = te1.pivot_table(index = "nome_x", values = "capacidade_aluno", aggfunc = len)

In [84]:
#Juntando os dataframes depois de serem tratados
teste = pd.concat([itp, ptp], ignore_index = True)